In [ ]:
#@title Traducción y análisis de sentimientos

# ============================================================
# COLAB: Traducción por país + Sentiment EN
# CON MENSAJES DE PROGRESO
# ============================================================

!pip -q install pandas numpy transformers sentencepiece accelerate

import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification

print("\n[INFO] Inicio del pipeline")

# ----------------------------
# 1) Cargar dataset
# ----------------------------
PATH = "replies_meta_desde_urls.csv"
df = pd.read_csv(PATH, sep=";", encoding="latin1", engine="python")

df["text"] = df["text"].astype(str).fillna("").str.replace(r"\s+", " ", regex=True).str.strip()
df["_user_original_norm"] = df["user_original"].astype(str).str.lower().str.strip()

media_to_country = {
    "@20m": "España",
    "@el_pais": "España",
    "@eldiarioes": "España",
    "@elmundoes": "España",
    "@ensonhaber": "Turquía",
    "@gazetesozcu": "Turquía",
    "@haberturk": "Turquía",
    "@nypost": "Estados Unidos",
    "@nytimes": "Estados Unidos",
    "@washingtonpost": "Estados Unidos",
    "@wsj": "Estados Unidos",
    "@usatoday": "Estados Unidos",
}
df["pais"] = df["_user_original_norm"].map(media_to_country)

mask_nonempty = df["text"].ne("")
print(f"[INFO] Total filas con texto: {mask_nonempty.sum()}")

print("\n[INFO] Distribución por país:")
print(df["pais"].value_counts(dropna=False))

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n[INFO] Device: {device}")

# ----------------------------
# 2) Traducción (solo España y Turquía)
# ----------------------------
print("\n[INFO] Cargando modelo de traducción (NLLB)...")
TRANSLATION_MODEL = "facebook/nllb-200-distilled-600M"
trans_tokenizer = AutoTokenizer.from_pretrained(TRANSLATION_MODEL)
trans_model = AutoModelForSeq2SeqLM.from_pretrained(TRANSLATION_MODEL).to(device)

@torch.inference_mode()
def translate_to_english(texts, src_lang_code, label, batch_size=16, max_length=256):
    trans_tokenizer.src_lang = src_lang_code
    outputs = []
    total = len(texts)
    n_batches = (total + batch_size - 1) // batch_size

    for i in range(0, total, batch_size):
        b = i // batch_size + 1
        print(f"[TRADUCCIÓN {label}] Lote {b}/{n_batches}")
        batch = texts[i:i+batch_size]

        enc = trans_tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        ).to(device)

        gen = trans_model.generate(
            **enc,
            forced_bos_token_id=trans_tokenizer.convert_tokens_to_ids("eng_Latn"),
            max_new_tokens=max_length,
            num_beams=4,
        )
        outputs.extend(trans_tokenizer.batch_decode(gen, skip_special_tokens=True))

    print(f"[TRADUCCIÓN {label}] Finalizada")
    return outputs

df["text_en"] = ""

# EEUU → copiar
mask_us = mask_nonempty & (df["pais"] == "Estados Unidos")
print(f"\n[INFO] EEUU → no se traduce: {mask_us.sum()} textos")
df.loc[mask_us, "text_en"] = df.loc[mask_us, "text"]

# España → traducir
mask_es = mask_nonempty & (df["pais"] == "España")
print(f"[INFO] España → se traducen: {mask_es.sum()} textos")
if mask_es.any():
    df.loc[mask_es, "text_en"] = translate_to_english(
        df.loc[mask_es, "text"].tolist(), "spa_Latn", label="ES"
    )

# Turquía → traducir
mask_tr = mask_nonempty & (df["pais"] == "Turquía")
print(f"[INFO] Turquía → se traducen: {mask_tr.sum()} textos")
if mask_tr.any():
    df.loc[mask_tr, "text_en"] = translate_to_english(
        df.loc[mask_tr, "text"].tolist(), "tur_Latn", label="TR"
    )

# Fallback
mask_left = mask_nonempty & df["text_en"].eq("")
df.loc[mask_left, "text_en"] = df.loc[mask_left, "text"]

print("\n[INFO] Traducción completada")

# ----------------------------
# 3) Sentiment en inglés
# ----------------------------
print("\n[INFO] Cargando modelo de sentimiento...")
SENT_MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sent_tokenizer = AutoTokenizer.from_pretrained(SENT_MODEL)
sent_model = AutoModelForSequenceClassification.from_pretrained(SENT_MODEL).to(device)

labels = {int(k): v.lower() for k, v in sent_model.config.id2label.items()}
neg_i = [k for k,v in labels.items() if "neg" in v][0]
pos_i = [k for k,v in labels.items() if "pos" in v][0]

@torch.inference_mode()
def sentiment_probs(texts, batch_size=32, max_length=128):
    outputs = []
    total = len(texts)
    n_batches = (total + batch_size - 1) // batch_size

    for i in range(0, total, batch_size):
        b = i // batch_size + 1
        print(f"[SENTIMENT] Lote {b}/{n_batches}")
        batch = texts[i:i+batch_size]

        enc = sent_tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length,
        ).to(device)

        logits = sent_model(**enc).logits
        outputs.append(torch.softmax(logits, dim=1).detach().cpu().numpy())

    print("[SENTIMENT] Finalizado")
    return np.vstack(outputs)

texts_en = df["text_en"].astype(str).tolist()
probs = sentiment_probs(texts_en)

df["polaridad"] = probs[:, pos_i] - probs[:, neg_i]

print("\n[INFO] Polaridad media por país:")
print(df.groupby("pais")["polaridad"].mean())

# ----------------------------
# 4) Guardar
# ----------------------------
out_path = "replies_meta_en_sentiment.csv"
df.drop(columns=["_user_original_norm"], errors="ignore").to_csv(out_path, index=False, encoding="utf-8")
print(f"\n[OK] Archivo guardado: {out_path}")


In [ ]:
# ============================================================
#@title GRÁFICOS (desde replies_meta_en_sentiment.csv)
#   - Lee el CSV ya procesado (sin volver a traducir)
#   - Genera y guarda PNGs (300 dpi) listos para paper
# ============================================================

!pip -q install pandas numpy matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# 1) Cargar resultados
# ----------------------------
PATH_OUT = "replies_meta_en_sentiment.csv"
df = pd.read_csv(PATH_OUT, encoding="utf-8")

# Asegura tipos
df["polaridad"] = pd.to_numeric(df["polaridad"], errors="coerce")
df["pais"] = df["pais"].astype(str)

# Filtrado básico
dfp = df.dropna(subset=["polaridad", "pais"]).copy()
dfp = dfp[dfp["polaridad"].between(-1, 1, inclusive="both")]

# Normaliza medio
dfp["medio_norm"] = dfp["user_original"].astype(str).str.strip().str.lower()

# Orden fijo de países
country_order = ["España", "Estados Unidos", "Turquía"]
country_order = [c for c in country_order if c in dfp["pais"].unique()]

print("N por país:")
print(dfp["pais"].value_counts())

# ----------------------------
# 2) BOXplot polaridad por país (recomendado)
# ----------------------------
data_by_country = [dfp.loc[dfp["pais"] == c, "polaridad"].values for c in country_order]

plt.figure(figsize=(8, 5))
plt.boxplot(data_by_country, labels=country_order, showfliers=False)
plt.axhline(0, linewidth=1)
plt.ylabel("Polaridad (p_pos − p_neg)")
plt.title("Distribución de la polaridad de las respuestas por país")
plt.tight_layout()
plt.savefig("fig1_boxplot_polaridad_pais.png", dpi=300)
plt.show()

# ----------------------------
# 3) Histograma/densidad por país (superpuesto)
# ----------------------------
plt.figure(figsize=(8, 5))
for c in country_order:
    vals = dfp.loc[dfp["pais"] == c, "polaridad"].values
    plt.hist(vals, bins=30, alpha=0.5, density=True, label=c)
plt.axvline(0, linewidth=1)
plt.xlabel("Polaridad (p_pos − p_neg)")
plt.ylabel("Densidad")
plt.title("Distribución de polaridad por país")
plt.legend()
plt.tight_layout()
plt.savefig("fig2_hist_polaridad_pais.png", dpi=300)
plt.show()

# ----------------------------
# 4) Barras: polaridad media por país (con N)
# ----------------------------
means = dfp.groupby("pais")["polaridad"].mean().reindex(country_order)
counts = dfp.groupby("pais")["polaridad"].count().reindex(country_order)

plt.figure(figsize=(7, 5))
plt.bar(means.index, means.values)
plt.axhline(0, linewidth=1)
plt.ylabel("Polaridad media")
plt.title("Polaridad media de respuestas por país")
# Anotar N encima
for i, c in enumerate(means.index):
    plt.text(i, means.values[i], f"N={int(counts[c])}", ha="center", va="bottom")
plt.tight_layout()
plt.savefig("fig3_media_polaridad_pais.png", dpi=300)
plt.show()

# ----------------------------
# 5) Polaridad media por medio (Top/Bottom N)
# ----------------------------
N = 10  # ajusta a 5/10/15
means_media = dfp.groupby("medio_norm")["polaridad"].mean().sort_values()
bottom = means_media.head(N)
top = means_media.tail(N)
combo = pd.concat([bottom, top])

plt.figure(figsize=(10, 6))
plt.barh(combo.index, combo.values)
plt.axvline(0, linewidth=1)
plt.xlabel("Polaridad media")
plt.title(f"Polaridad media por medio (Bottom {N} y Top {N})")
plt.tight_layout()
plt.savefig("fig4_media_polaridad_medio_topbottom.png", dpi=300)
plt.show()

# ----------------------------
# 6) Boxplot por medio (solo medios con N mínimo)
# ----------------------------
MIN_N = 30  # cambia a 20/30/50
counts_media = dfp["medio_norm"].value_counts()
keep = counts_media[counts_media >= MIN_N].index.tolist()
dfm = dfp[dfp["medio_norm"].isin(keep)].copy()

# Orden de medios por media
order_media = dfm.groupby("medio_norm")["polaridad"].mean().sort_values().index.tolist()
data_by_media = [dfm.loc[dfm["medio_norm"] == m, "polaridad"].values for m in order_media]

plt.figure(figsize=(12, 6))
plt.boxplot(data_by_media, labels=order_media, showfliers=False)
plt.axhline(0, linewidth=1)
plt.ylabel("Polaridad (p_pos − p_neg)")
plt.title(f"Distribución de polaridad por medio (N≥{MIN_N})")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig("fig5_boxplot_polaridad_medio.png", dpi=300)
plt.show()

# ----------------------------
# 7) (Recomendado) Versión "sin Turquía" para paper (si N es muy bajo)
# ----------------------------
df_no_tr = dfp[dfp["pais"] != "Turquía"].copy()
country_order_no_tr = [c for c in ["España", "Estados Unidos"] if c in df_no_tr["pais"].unique()]
data_no_tr = [df_no_tr.loc[df_no_tr["pais"] == c, "polaridad"].values for c in country_order_no_tr]

plt.figure(figsize=(7, 5))
plt.boxplot(data_no_tr, labels=country_order_no_tr, showfliers=False)
plt.axhline(0, linewidth=1)
plt.ylabel("Polaridad (p_pos − p_neg)")
plt.title("Distribución de la polaridad (España vs EEUU)")
plt.tight_layout()
plt.savefig("fig6_boxplot_polaridad_pais_sin_turquia.png", dpi=300)
plt.show()

print("\n✅ Figuras guardadas:")
print([
    "fig1_boxplot_polaridad_pais.png",
    "fig2_hist_polaridad_pais.png",
    "fig3_media_polaridad_pais.png",
    "fig4_media_polaridad_medio_topbottom.png",
    "fig5_boxplot_polaridad_medio.png",
    "fig6_boxplot_polaridad_pais_sin_turquia.png",
])

# Para descargar en Colab (opcional):
# from google.colab import files
# for f in [
#   "fig1_boxplot_polaridad_pais.png",
#   "fig2_hist_polaridad_pais.png",
#   "fig3_media_polaridad_pais.png",
#   "fig4_media_polaridad_medio_topbottom.png",
#   "fig5_boxplot_polaridad_medio.png",
#   "fig6_boxplot_polaridad_pais_sin_turquia.png",
# ]:
#     files.download(f)
